<a href="https://colab.research.google.com/drive/1IxiJ0fjBvuVslq86bR-WogzvKGoVYlJz">Abre este Jupyter en Google Colab</a>

# Evaluación del caso Titanic mediante _Support Vector Machine (SVM)_

### Descripción
El hundimiento del Titanic es uno de los naufragios más tristemente célebres de la historia.

El 15 de abril de 1912, durante su viaje inaugural, el considerado "insumergible" RMS Titanic se hundió tras chocar con un iceberg. Desgraciadamente, no había suficientes botes salvavidas para todos los ocupantes, lo que provocó la muerte de 1.502 de los 2.224 pasajeros y tripulantes.

Aunque hubo un elemento de suerte en la supervivencia, parece que algunos grupos de personas tenían más probabilidades de sobrevivir que otros.

En este reto, te pedimos que construyas un modelo predictivo que responda a la pregunta: "¿qué tipo de personas tenían más probabilidades de sobrevivir?" utilizando los datos de los pasajeros (es decir, nombre, edad, sexo, clase socioeconómica, etc.).


## Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## Cargamos los datos de entrenamiento

In [2]:
train_data = pd.read_csv('train.csv')

## Eliminamos columnas irrelevantes o con muchos datos faltantes

In [3]:
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

## Limpiamos los datos: completamos valores faltantes con la mediana de cada columna

In [ ]:
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)

## Convertimos variables categóricas a variables dummy

In [5]:
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'])

## Separamos features y etiquetas

In [6]:
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

## Normalizamos los datos

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Dividimos los datos en conjunto de entrenamiento y en conjunto de prueba

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Definimos los parámetros a ajustar

In [9]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 10],
    'kernel': ['rbf']
}

## Creamos un modelo SVM

In [10]:
svm_model = SVC(random_state=42)

## Ajustamos hiperparámetros utilizando GridSearchCV

In [11]:
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['rbf']},
             scoring='accuracy')

## Obtenemos los mejores hiperparámetros

In [12]:
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


## Entrenamos el modelo SVM con los mejores hiperparámetros

In [13]:
best_svm_model = SVC(**best_params, random_state=42)
best_svm_model.fit(X_train, y_train)

SVC(C=10, gamma=0.1, random_state=42)

## Predecimos sobre el conjunto de prueba

In [14]:
y_pred = best_svm_model.predict(X_test)

## Calculamos la precisión

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo SVM en el conjunto de prueba:", accuracy)

Precisión del modelo SVM en el conjunto de prueba: 0.8156424581005587


## Mostramos el informe de clasificación

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       105
           1       0.83      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.80       179
weighted avg       0.82      0.82      0.81       179



## Cargamos los datos de prueba

In [17]:
test_data = pd.read_csv('test.csv')

### Guardamos los IDs de los pasajeros para la presentación final

In [18]:
passenger_ids = test_data['PassengerId']

### Eliminamos columnas irrelevantes o con muchos datos faltantes

In [19]:
test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

### Limpiamos datos: se completan los valores faltantes con la mediana de cada columna

In [ ]:
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

### Convertimos variables categóricas a variables dummy

In [21]:
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked'])

### Normalizamos datos de prueba utilizando el mismo StandardScaler que utilizamos para los datos de entrenamiento

In [22]:
X_test_scaled = scaler.transform(test_data)

### Hacemos predicciones sobre los datos de prueba

In [23]:
svm_model.fit(X_train, y_train)
predictions = svm_model.predict(X_test_scaled)

### Creamos un DataFrame con los resultados

In [24]:
results = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': predictions})

### Guardamos los resultados en un archivo CSV para la presentación en Kaggle

In [25]:
results.to_csv('titanic_predictions.csv', index=False)

In [27]:
# Cargar el archivo CSV
predictions_df = pd.read_csv('titanic_predictions.csv')

# Mostrar las primeras filas para verificar la estructura del archivo
print(predictions_df.head(30))


    PassengerId  Survived
0           892         0
1           893         0
2           894         0
3           895         0
4           896         0
5           897         0
6           898         1
7           899         0
8           900         1
9           901         0
10          902         0
11          903         0
12          904         1
13          905         0
14          906         1
15          907         1
16          908         0
17          909         0
18          910         0
19          911         1
20          912         0
21          913         1
22          914         1
23          915         0
24          916         1
25          917         0
26          918         1
27          919         0
28          920         0
29          921         0
